In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
torch.manual_seed(42)

In [5]:
df = pd.read_csv('/content/drive/MyDrive/fashion-mnist_train.csv')
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
x = df.iloc[:, 1:].values
y = df.iloc[:,0].values

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [8]:
# transformation
from torchvision.transforms import transforms

custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
from PIL import Image
import numpy as np

class CustomDataset(Dataset):
  def __init__(self, features, labels, transforms):
    self.features = features
    self.labels = labels
    self.transforms = transforms

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    # resize to 28,28
    img = self.features[idx].reshape(28,28)

    # change dtype to np.uint8
    img = img.astype(np.uint8)

    # change Black&white to color image --> (C,H,W) -> (H,W,C)
    img = np.stack([img]*3, axis=-1)

    # convert array to PIL image
    img = Image.fromarray(img)

    # apply transforms
    img = self.transforms(img)

    return img, torch.tensor(self.labels[idx], dtype=torch.long)

In [10]:
# example of np.stack
arr = [[1,2],[3,4]]
arr

[[1, 2], [3, 4]]

In [11]:
np.stack([arr]*3)

array([[[1, 2],
        [3, 4]],

       [[1, 2],
        [3, 4]],

       [[1, 2],
        [3, 4]]])

In [12]:
train_dataset = CustomDataset(x_train, y_train, transforms=custom_transform)
test_dataset = CustomDataset(x_test, y_test, transforms=custom_transform)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [14]:
# fetch the pretrained model
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:01<00:00, 498MB/s]


In [15]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
# dont transform the feature part just chhed chhad only classifier part

In [17]:
for params in vgg16.features.parameters():
  params.required_grad=False

In [18]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),

    nn.Linear(512, 10),
)

In [19]:
vgg16 = vgg16.to(device)

In [20]:
learning_rate = 0.0001
epochs = 10

In [21]:
# loss function
criterian = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0
  for batch_features, batch_label in train_loader:

    # move to GPU
    batch_features = batch_features.to(device)
    batch_label = batch_label.to(device)

    # forward
    out = vgg16(batch_features)

    # loss calculation
    loss = criterian(out, batch_label)

    # back prop
    optimizer.zero_grad()
    loss.backward()

    # update grad
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)

  print(f"epoch: {epoch+1}, loss: {avg_loss}")

In [ ]:
vgg16.eval()

In [ ]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_label in test_loader:
    # move to GPU
    batch_features = batch_features.to(device)
    batch_label = batch_label.to(device)

    out = vgg16(batch_features)

    _, pred = torch.max(out.data, 1)

    total = total + batch_label.shape[0]
    correct = correct + (pred == batch_label).sum().item()

print(f" accuracy : {(correct/total)*100}%")

TAKING TO MUCH TIME TO RUN EPOCHS.   
ACCURACY CAME OUT TO BE 94%